# Import Libary Package

In [ ]:
!pip install -qU py_vncorenlp emot levenshtein tqdm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import json
import re
from difflib import SequenceMatcher
import itertools
from itertools import combinations, product, chain
from pprint import pprint
from ast import literal_eval
from tqdm import tqdm

from emot.emo_unicode import UNICODE_EMOJI
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from Levenshtein import ratio

from __future__ import unicode_literals
import operator
import sys

try:
    from html.parser import HTMLParser
except ImportError:
    from HTMLParser import HTMLParser

try:
    import html
except ImportError:
    pass
import py_vncorenlp

# NER Pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

ner = pipeline("ner", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/411k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [ ]:
def replace_person_with_username(sentence, ner_results):
  """
  Replaces all PERSON entities in a sentence with "@username".

  Args:
    sentence: The sentence to process.
    ner_results: The NER results for the sentence.

  Returns:
    The sentence with PERSON entities replaced with "@username".
  """

  for entity in ner_results:
    if entity["entity_group"] == "PERSON":
      sentence = sentence.replace(entity["word"], "@username")

  return sentence

In [ ]:
sentence = "@Nguyễn Thiện dô đây xem thử đi ạ"
ner_results = ner(sentence, aggregation_strategy='simple')

replaced_sentence = replace_person_with_username(sentence, ner_results)

print(replaced_sentence)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


@@username dô đây xem thử đi ạ


# Preprocess Fuction

In [ ]:
def regex_or(*items):
    return '(?:' + '|'.join(items) + ')'

In [ ]:
punctChars = r"['\"“”‘’.?!…,:;]"
#punctSeq   = punctChars+"+"	#'anthem'. => ' anthem '.
punctSeq   = r"['\"“”‘’]+|[.?!,…]+|[:;]+"	#'anthem'. => ' anthem ' .
entity     = r"&(?:amp|lt|gt|quot);"

#URLs
urlStart1  = r"(?:https?://|\bwww\.)"
commonTLDs = r"(?:com|org|edu|gov|net|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|pro|tel|travel|xxx)"
ccTLDs	 = r"(?:ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|" + \
r"bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|" + \
r"er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|" + \
r"hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|" + \
r"lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|" + \
r"nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|sk|" + \
r"sl|sm|sn|so|sr|ss|st|su|sv|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|" + \
r"va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|za|zm|zw)"	#TODO: remove obscure country domains?
urlStart2  = r"\b(?:[A-Za-z\d-])+(?:\.[A-Za-z0-9]+){0,3}\." + regex_or(commonTLDs, ccTLDs) + r"(?:\."+ccTLDs+r")?(?=\W|$)"
urlBody    = r"(?:[^\.\s<>][^\s<>]*?)?"
urlExtraCrapBeforeEnd = regex_or(punctChars, entity) + "+?"
urlEnd     = r"(?:\.\.+|[<>]|\s|$)"
url        = regex_or(urlStart1, urlStart2) + urlBody + "(?=(?:"+urlExtraCrapBeforeEnd+")?"+urlEnd+")"

# Numeric
timeLike   = r"\d+(?::\d+){1,2}"  # (1:00:00)
numdotNum = r"\d+\.\d+"
rationum = r"\d+\-\d+"
numberWithCommas = r"(?:(?<!\d)\d{1,3},)+?\d{3}" + r"(?=(?:[^,\d]|$))"  #(1,5  2,5)
#currency
numComb	 = u"[\u0024\u058f\u060b\u09f2\u09f3\u09fb\u0af1\u0bf9\u0e3f\u17db\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6\u00a2-\u00a5\u20a0-\u20b9]?\\d+(?:\\.\\d+)+%?"

boundaryNotDot = regex_or("$", r"\s", r"[“\"?!,:;]", entity)
separators  = "(?:--+|―|—|~|–|=)"
decorations = u"(?:[♫♪]+|[★☆]+|[♥❤♡]+|[\u2639-\u263b]+|[\ue001-\uebbb]+)"
thingsThatSplitWords = r"[^\s\.,?\"]"


#  Emoticons
# myleott: in Python the (?iu) flags affect the whole expression
#normalEyes = "(?iu)[:=]" # 8 and x are eyes but cause problems
normalEyes = "[:=]" # 8 and x are eyes but cause problems
wink = "[;]"
noseArea = "(?:|-|[^a-zA-Z0-9 ])" # doesn't get :'-(
happyMouths = r"[D\)\]\}\>]+"
sadMouths = r"[\(\[\{\<]+"
tongue = "[pPd3*]+"
otherMouths = r"(?:[oO]+|[/\\]+|[vV]+|[Ss]+|[|]+)" # remove forward slash if http://'s aren't cleaned
bfLeft = u"(♥|0|[oO]|°|[vV]|\\$|[tT]|[xX]|;|\u0ca0|@|ʘ|•|・|◕|\\^|¬|\\*)"
bfCenter = r"(?:[\.]|[_-]+)"
bfRight = r"\2"
s3 = r"(?:--['\"])"
s4 = r"(?:<|&lt;|>|&gt;)[\._-]+(?:<|&lt;|>|&gt;)"
s5 = "(?:[.][_]+[.])"

double_eyes = r"(?:[^]" + r"[^])"
basicface = "(?:" +bfLeft+bfCenter+bfRight+ ")|" +s3+ "|" +s4+ "|" + s5
eeLeft = r"[＼\\ƪԄ\(（<>;ヽ\-=~\*]+"
eeRight= u"[\\-=\\);'\u0022<>ʃ）/／ノﾉ丿╯σっµ~\\*]+"
eeSymbol = r"[^A-Za-z0-9\s\(\)\*:=-]"
eastEmote = eeLeft + "(?:"+basicface+"|" +eeSymbol+")+" + eeRight
oOEmote = r"(?:[oO]" + bfCenter + r"[oO])"

Hearts = "(?:<+/?3+)+"
Arrows = regex_or(r"(?:<*[-―—=]*>+|<+[-―—=]*>*)", u"[\u2190-\u21ff]+")
Hashtag = "#[a-zA-Z0-9_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ]+"
AtMention = "[@＠][a-zA-Z0-9_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ]+"
Bound = r"(?:\W|^|$)"
Email = regex_or("(?<=(?:\W))", "(?<=(?:^))") + r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4}(?=" +Bound+")"

# Edge punctuation
edgePunctChars    = u"'\"“”‘’«»{}\\(\\)\\[\\]\\*&" #add \\p{So}? (symbols)
edgePunct    = "[" + edgePunctChars + "]"
notEdgePunct = "[a-zA-Z0-9_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ]" # content characters
offEdge = r"(^|$|:|;|\s|\.|,)"  # colon here gets "(hello):" ==> "( hello ):"
EdgePunctLeft  = re.compile(offEdge + "("+edgePunct+"+)("+notEdgePunct+")", re.UNICODE)
EdgePunctRight = re.compile("("+notEdgePunct+")("+edgePunct+"+)" + offEdge, re.UNICODE)

In [ ]:
emoticon = regex_or(
        # Standard version  :) :( :] :D :P
        "(?:>|&gt;)?" + regex_or(normalEyes, wink) + regex_or(noseArea,"[Oo]") + regex_or(tongue+r"(?=\W|$|RT|rt|Rt)", otherMouths+r"(?=\W|$|RT|rt|Rt)", sadMouths, happyMouths),

        # reversed version (: D:  use positive lookbehind to remove "(word):"
        # because eyes on the right side is more ambiguous with the standard usage of : ;
        regex_or("(?<=(?: ))", "(?<=(?:^))") + regex_or(sadMouths,happyMouths,otherMouths) + noseArea + regex_or(normalEyes, wink) + "(?:<|&lt;)?",

        #inspired by http://en.wikipedia.org/wiki/User:Scapler/emoticons#East_Asian_style
        eastEmote.replace("2", "1", 1), basicface,
        # iOS 'emoji' characters (some smileys, some symbols) [\ue001-\uebbb]
        # TODO should try a big precompiled lexicon from Wikipedia, Dan Ramage told me (BTO) he does this

        # myleott: o.O and O.o are two of the biggest sources of differences
        #          between this and the Java version. One little hack won't hurt...
        oOEmote
)

In [ ]:
# m/d/yyyy + mm/dd/yyyy
d1 = r"^(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])/(?:[0-9]{2})?[0-9]{2}$"
# d/m/yyyy + dd/mm/yyyy
d2 = r"^(3[01]|[12][0-9]|0?[1-9])/(1[0-2]|0?[1-9])/(?:[0-9]{2})?[0-9]{2}$"
# m/d or mm/dd
d3 = r"^(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])"
# d/m or dd/mm
d4 = r"^(3[01]|[12][0-9]|0?[1-9])/(1[0-2]|0?[1-9])"

date = regex_or(d1,d2,d3,d4)

In [ ]:
# We will be tokenizing using these regexps as delimiters
# Additionally, these things are "protected", meaning they shouldn't be further split themselves.
Protected  = re.compile(
    regex_or(
        Hearts,
        url,
        Email,
        timeLike,
        date,
        numdotNum,
        rationum,
        numberWithCommas,
        numComb,
        emoticon,
        Arrows,
        entity,
        punctSeq,
        separators,
        decorations,
        Hashtag,
        AtMention), re.UNICODE)

In [ ]:
emoji_list = list(UNICODE_EMOJI)

In [ ]:
def remove_dotNotend_in_list(lst):
  for i, e in enumerate(lst):
    if e == '.':
      if (i != len(lst) -1) and (i == 0 or not lst[i + 1].isupper()):
        lst[i] = ''
      else:
        pass
  return lst

In [ ]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251"
    "]+", flags=re.UNICODE)

In [ ]:
def splitEdgePunct(input):
    input = EdgePunctLeft.sub(r"\1\2 \3", input)
    input = EdgePunctRight.sub(r"\1 \2\3", input)
    return input

def addAllnonempty(master, smaller):
    for s in smaller:
        strim = s.strip()
        if (len(strim) > 0):
            master.append(strim)
    return master

def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

def split_emoji_text(text):
    result = ''.join((' {} '.format(el) if el in emoji_list else el for el in text))
    return result


def split_emoji_emoji(textArray):
    result = []
    for word in textArray:
        is_emoji = False

        # iterate element of characters and check char is emoji if yes
        # then append into result list and set is_emogi flag true.
        for char in word:
            if char in [*emoji_list]:
                result.append(char)
                is_emoji = True

        # check is_emoji flag is false then append string into list.
        if not is_emoji:
            result.append(word)

    return result

def wordSegmentandKeepicon(text):

    # Do the no-brainers first
    splitPunctText = splitEdgePunct(text)

    textLength = len(splitPunctText)

    # BTO: the logic here got quite convoluted via the Scala porting detour
    # It would be good to switch back to a nice simple procedural style like in the Python version
    # ... Scala is such a pain.  Never again.

    # Find the matches for subsequences that should be protected,
    # e.g. URLs, 1.0, U.N.K.L.E., 12:53
    bads = []
    badSpans = []
    for match in Protected.finditer(splitPunctText):
        # The spans of the "bads" should not be split.
        if (match.start() != match.end()): #unnecessary?
            bads.append( [splitPunctText[match.start():match.end()]] )
            badSpans.append( (match.start(), match.end()) )
    for match in emoji_pattern.finditer(splitPunctText):
        if match.start() != match.end():
            bads.append([splitPunctText[match.start():match.end()]])
            badSpans.append((match.start(), match.end()))

    # Create a list of indices to create the "goods", which can be
    # split. We are taking "bad" spans like
    #     List((2,5), (8,10))
    # to create
    #     List(0, 2, 5, 8, 10, 12)
    # where, e.g., "12" here would be the textLength
    # has an even length and no indices are the same
    indices = [0]
    for (first, second) in badSpans:
        indices.append(first)
        indices.append(second)
    indices.append(textLength)

    # Group the indices and map them to their respective portion of the string
    splitGoods = []
    for i in range(0, len(indices), 2):
        goodstr = splitPunctText[indices[i]:indices[i+1]]
        goodstr = rdrsegmenter.word_segment(goodstr)
        #splitstr = goodstr.strip().split(" ")
        splitGoods.append(goodstr)


    #  Reinterpolate the 'good' and 'bad' Lists, ensuring that
    #  additonal tokens from last good item get included
    zippedStr = []
    for i in range(len(bads)):
        zippedStr = addAllnonempty(zippedStr, splitGoods[i])
        zippedStr = addAllnonempty(zippedStr, bads[i])
    zippedStr = addAllnonempty(zippedStr, splitGoods[len(bads)])

    # BTO: our POS tagger wants "ur" and "you're" to both be one token.
    # Uncomment to get "you 're"
    #splitStr = []
    #for tok in zippedStr:
    #    splitStr.extend(splitToken(tok))
    #zippedStr = splitStr

    return zippedStr

def preprocess(sentence):
    #text = replace_all(text, dict_map)
    ner_results = ner(sentence,aggregation_strategy='simple')
    text = replace_person_with_username(sentence, ner_results)


    text = split_emoji_text(text)
    text = wordSegmentandKeepicon(text)
    text = remove_dotNotend_in_list(text)
    text = split_emoji_emoji(text)
    text = list(filter(str.strip, text))

    return ' '.join(text)

# Word Segmentation

In [ ]:
# Automatically download VnCoreNLP components from the original repository
# and save them in some local machine folder
py_vncorenlp.download_model(save_dir='/content')

In [ ]:
# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

In [ ]:
text = 'Hạnh Nguyễn con mẹ nó đoạn hôm qua t làm rồi nhưng mà nay lỗi'
output = preprocess(text)
print("Orignal:", text)
print("Segmented:", output)

Orignal: Hạnh Nguyễn con mẹ nó đoạn hôm qua t làm rồi nhưng mà nay lỗi
Segmented: @username con_mẹ nó đoạn hôm_qua t làm rồi nhưng_mà nay lỗi


# Load data

In [ ]:
def split_sentence_by_punctuation(text):
    # Define a regular expression pattern to match punctuation marks
    pattern = r'([.!?{\n}]{1,6})'

    # Use the re.split() function with the pattern
    splits = re.split(pattern, text)

    # Combine adjacent punctuation marks with their preceding split
    # This is done to ensure that each split contains both the punctuation mark and the preceding text
    result = []

    for pair in zip(splits[::2], splits[1::2]):
        result.append("".join(pair))
    if splits[-1]:
        result.append(splits[-1])

    return result

In [ ]:
def split_paragraph(text):
    # Define a regular expression pattern to match newline characters
    newline_pattern = r'\n+'

    # Split the text by newline characters
    splits = re.split(newline_pattern, text)

    # Remove empty strings from the splits
    splits = [split for split in splits if split.strip()]

    return splits

In [ ]:
def remove_url(text):
    # Define a regular expression pattern to match URLs
    url_pattern = r'https?://\S+|www\.\S+'

    # Find all URLs in the text
    urls = re.findall(url_pattern, text)

    # Split the text by the URLs
    splits = re.split(url_pattern, text)

    # Combine splits and URLs into a single list
    result = []
    for split, url in zip(splits, urls):
        result.append(split)
        #result.append(url)

    # Add the last split (if any) to the result
    if splits[-1]:
        result.append(splits[-1])

    return " ".join(result)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Tokenize data and create dataset

In [ ]:
vilexnorm_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViLexNorm.csv"
vihsd_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViHSD.csv"
vsmec_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/VSMEC.csv"
visfd_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViSFD.csv"
vihos_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViHOS.csv"
vispamreviews_path = "/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViSpamReviews.csv"

In [ ]:
def read_dataset(path):
  data = pd.read_csv(path)
  data = data.dropna().reset_index(drop=True)
  data['index'] = data.index
  return data

def processing_downstream_dataset(data, name):
  url_pattern = r'https?://\S+|www\.\S+'
  df = pd.DataFrame(columns=['dataset', 'type','sent_idx', 'idx', 'original', 'input'])

  for i in tqdm(range(0,len(data)), desc ="Proccessed:"):
      if re.findall(url_pattern, data[name][i]) != []:
          sent = split_paragraph(remove_url(data[name][i]))
      elif len(data[name][i]) > 128:
          sent = split_sentence_by_punctuation(data[name][i])
      else:
          sent = [data[name][i]]

      for j in range(len(sent)):
          text = preprocess(sent[j])

          df.loc[len(df)] = {'dataset': data['dataset'][i],
                          'type': data['type'][i],
                          'sent_idx': i,
                          'idx': j,
                          'original': text,
                          'input': [w.replace('_', ' ') for w in text.split()]}
  df['original'] = df['original'].str.replace('_', ' ')
  return df

In [ ]:
vihsd = read_dataset(vihsd_path) #free_text
vsmec = read_dataset(vsmec_path) #Sentence
visfd = read_dataset(visfd_path) #comment
vihos = read_dataset(vihos_path) #sentence
vispamreviews = read_dataset(vispamreviews_path) #Comment

In [ ]:
df_vihsd = processing_downstream_dataset(vihsd, 'free_text')
df_vsmec = processing_downstream_dataset(vsmec, 'Sentence')
df_visfd = processing_downstream_dataset(visfd, 'comment')
df_vihos = processing_downstream_dataset(vihos, 'sentence')
df_vispamreviews = processing_downstream_dataset(vispamreviews, 'Comment')

Proccessed:: 100%|██████████| 19870/19870 [1:39:11<00:00,  3.34it/s]


In [ ]:
data = pd.concat([df_vihsd,df_vsmec, df_vihos, df_visfd, df_vispamreviews]).reset_index(drop=True)
data

,dataset,type,sent_idx,idx,original,input
0,ViHSD,train,0,0,Em được làm fan cứng luôn rồi nè ❤ ️ reaction ...,"[Em, được, làm, fan, cứng, luôn, rồi, nè, ❤, ️..."
1,ViHSD,train,1,0,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,"[Đúng, là, bọn, mắt, híp, lò xo, thụt, :))), b..."
2,ViHSD,train,2,0,@username giờ giống thằng sida hơn à,"[@username, giờ, giống, thằng, sida, hơn, à]"
3,ViHSD,train,3,0,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN . NHÀ N...,"[CÔN ĐỒ, CỤC SÚC, VÔ NHÂN, TÍNH, ĐỀ, NGHI, VN,..."
4,ViHSD,train,4,0,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,"[Từ, lý thuyết, đến, thực hành, là, cả, 1, câu..."
...,...,...,...,...,...,...
121082,ViSpamReviews,test,19866,3,,[]
121083,ViSpamReviews,test,19866,4,Rêu có khả năng giữ nước và cho rễ cây bám hút...,"[Rêu, có, khả năng, giữ, nước, và, cho, rễ, câ..."
121084,ViSpamReviews,test,19867,0,Shop giao hàng nhanh nhưng mỗi tội cái shopee ...,"[Shop, giao, hàng, nhanh, nhưng, mỗi tội, cái,..."
121085,ViSpamReviews,test,19868,0,Shop ơi em có đặt 2 ram 8g dr4 nhưng máy em nh...,"[Shop, ơi, em, có, đặt, 2, ram, 8g, dr4, nhưng..."


In [ ]:
data.to_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/downstream_data.csv', index=False)

# read data

In [ ]:
from ast import literal_eval
dl = pd.read_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/ViLexNorm.csv' ,converters={"input": literal_eval})
dl

,Unnamed: 0,original,normalized,input,output,regrex_rule,dict_rule
0,0,thích anh cá mập k,thích anh cá mập không,"[thích, anh, cá, mập, k]","['thích', 'anh', 'cá', 'mập', 'không']","['thích', 'anh', 'cá', 'mập', 'không']","['thích', 'anh', 'quá', 'mập', 'không']"
1,1,cứ ngây thơ thế thoai :)),cứ ngây thơ thế thôi :)),"[cứ, ngây, thơ, thế, thoai, :))]","['cứ', 'ngây', 'thơ', 'thế', 'thôi', ':))']","['cứ', 'ngây', 'thơ', 'thế', 'thoai', ':))']","['cứ', 'ngây', 'thơ', 'thế', 'thôi', ':))']"
2,2,bà nghê xinh vậy mà t thấy k bằng bà chipu luô...,bà nghê xinh vậy mà tôi thấy không bằng bà chi...,"[bà, nghê, xinh, vậy, mà, t, thấy, k, bằng, bà...","['bà', 'nghê', 'xinh', 'vậy', 'mà', 'tôi', 'th...","['bà', 'nghê', 'xinh', 'vậy', 'mà', 't', 'thấy...","['ba', 'nghê', 'xinh', 'vậy', 'mà', 't', 'thấy..."
3,3,ê k khóc được làm thế nào má =)) ?,ê không khóc được làm thế nào má =)) ?,"[ê, k, khóc, được, làm, thế, nào, má, =)), ?]","['ê', 'không', 'khóc', 'được', 'làm', 'thế', '...","['ê', 'không', 'khóc', 'được', 'làm', 'thế', '...","['ê', 'không', 'khóc', 'được', 'là', 'thế', 'n..."
4,4,có biến gì hong dẫy :)),có biến gì không vậy :)),"[có, biến, gì, hong, dẫy, :))]","['có', 'biến', 'gì', 'không', 'vậy', ':))']","['có', 'biến', 'gì', 'hong', 'dẫy', ':))']","['có', 'biến', 'dì', 'hong', 'dẫy', ':))']"
...,...,...,...,...,...,...,...
10458,10458,từ lúc đu idol hàn và diễn viên thái thì hầu n...,từ lúc đu idol hàn và diễn viên thái thì hầu n...,"[từ, lúc, đu, idol, hàn, và, diễn, viên, thái,...","['từ', 'lúc', 'đu', 'idol', 'hàn', 'và', 'diễn...","['từ', 'lúc', 'đu', 'idol', 'hàn', 'và', 'diễn...","['từ', 'lúc', 'đu', 'idol', 'hàn', 'và', 'diễn..."
10459,10459,a hay nói e á,anh hay nói em á,"[a, hay, nói, e, á]","['anh', 'hay', 'nói', 'em', 'á']","['a', 'hay', 'nói', 'e', 'á']","['a', 'hay', 'nói', 'em', 'đá']"
10460,10460,đọc jd mà nó dễ quá đâm ra sợ cty lừa :))),đọc jd mà nó dễ quá đâm ra sợ công ty lừa :))),"[đọc, jd, mà, nó, dễ, quá, đâm, ra, sợ, cty, l...","['đọc', 'jd', 'mà', 'nó', 'dễ', 'quá', 'đâm', ...","['đọc', 'jd', 'mà', 'nó', 'dễ', 'quá', 'đâm', ...","['đọc', 'jd', 'mà', 'nó', 'dễ', 'quá', 'đâm', ..."
10461,10461,chỗ còn tuyển hong rủ tui vô làm nói chuyện dí...,chỗ còn tuyển không rủ tui vô làm nói chuyện v...,"[chỗ, còn, tuyển, hong, rủ, tui, vô, làm, nói,...","['chỗ', 'còn', 'tuyển', 'không', 'rủ', 'tui', ...","['chỗ', 'còn', 'tuyển', 'hong', 'rủ', 'tui', '...","['chỗ', 'càn', 'tuyển', 'hong', 'rũ', 'tui', '..."


In [ ]:
dl.iloc[10460].output

"['đọc', 'jd', 'mà', 'nó', 'dễ', 'quá', 'đâm', 'ra', 'sợ', 'công ty', 'lừa', ':)))']"

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/LexNorm/KLTN/Data/ASTRA/downstream_data.csv',converters={"input": literal_eval})
data

,dataset,type,sent_idx,idx,original,input
0,ViHSD,train,0,0,Em được làm fan cứng luôn rồi nè ❤ ️ reaction ...,"[Em, được, làm, fan, cứng, luôn, rồi, nè, ❤, ️..."
1,ViHSD,train,1,0,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,"[Đúng, là, bọn, mắt, híp, lò xo, thụt, :))), b..."
2,ViHSD,train,2,0,@username giờ giống thằng sida hơn à,"[@username, giờ, giống, thằng, sida, hơn, à]"
3,ViHSD,train,3,0,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN . NHÀ N...,"[CÔN ĐỒ, CỤC SÚC, VÔ NHÂN, TÍNH, ĐỀ, NGHI, VN,..."
4,ViHSD,train,4,0,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,"[Từ, lý thuyết, đến, thực hành, là, cả, 1, câu..."
...,...,...,...,...,...,...
121082,ViSpamReviews,test,19866,3,NaN,[]
121083,ViSpamReviews,test,19866,4,Rêu có khả năng giữ nước và cho rễ cây bám hút...,"[Rêu, có, khả năng, giữ, nước, và, cho, rễ, câ..."
121084,ViSpamReviews,test,19867,0,Shop giao hàng nhanh nhưng mỗi tội cái shopee ...,"[Shop, giao, hàng, nhanh, nhưng, mỗi tội, cái,..."
121085,ViSpamReviews,test,19868,0,Shop ơi em có đặt 2 ram 8g dr4 nhưng máy em nh...,"[Shop, ơi, em, có, đặt, 2, ram, 8g, dr4, nhưng..."


In [ ]:
data.iloc[76479].input

['Với',
 'pin',
 '5000',
 'mình',
 'có thể',
 'dùng',
 'cả',
 'ngày',
 ',',
 'không',
 'mất',
 'tg',
 'vừa',
 'dùng',
 'vừa',
 'xạc',
 '.']

In [ ]:
data.sample(3)

,dataset,type,sent_idx,idx,original,input
76480,UIT-ViSFD,dev,8451,1,Cảm biến vân tay trong màn hình thì phải mất 1...,"['Cảm biến', 'vân', 'tay trong', 'màn hình', '..."
101177,ViSpamReviews,train,9737,0,"Áo ok lắm nha sốp , mình thích mặc ôm chặt nên...","['Áo', 'ok', 'lắm', 'nha', 'sốp', ',', 'mình',..."
17741,ViHSD,train,16216,0,@username kia t còn chưa về nhà nữa ,"['@username', 'kia', 't', 'còn', 'chưa', 'về',..."
